# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

Name : Gaurav Jha
Roll No. : 19CH10074

In [42]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [44]:
#Load the IMDB dataset. You can load it using pandas as dataframe
df = pd.read_csv("IMDB_Dataset.csv",encoding = 'unicode_escape')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [45]:
#using lower cased corpus
df['review'] = df['review'].str.lower()
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [46]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
import string
from nltk.corpus import stopwords

#function to remove URLs from the text

def remove_URL(text):
    url = re.compile(r"https?://\S+www.\.\S+")
    return url.sub(r"",text)

#funtion to remove html content from the text

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"",text)

#function to remove punctuation from the text

def remove_punctuation(text):
    table = str.maketrans("","",string.punctuation)
    return text.translate(table)

#function to remove non alpha numbers from the text

def remove_non_alpha_numbers(text):
    txt = re.compile(r"\W+")
    return txt.sub(text," ")

# removing stopwords

stopwords = set(stopwords.words("english"))

def remove_stopwords(text):
    words = [x for x in text.split() if x not in stopwords]
    return " ".join(words)


# cleaning data


df['review'] = df['review'].map(remove_URL)
df['review'] = df['review'].map(remove_html)
df['review'] = df['review'].map(remove_punctuation)
df['review'] = df['review'].map(remove_stopwords)
df['review'] = df['review'].map(remove_alpha_numbers)
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive


In [53]:
# stemming the text data

dp = {}

from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stemming(text):
    stemmed_text = []
    for word in text.split():
        if word not in dp:
            dp[word] = stemmer.stem(word)
            stemmed_text.append(dp[word])
        else:
            stemmed_text.append(dp[word])
    return " ".join(stemmed_text)
df['review'] = df['review'].map(stemming)

df.head()

,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic there famili littl boy jake think there ...,negative
4,petter mattei love time money visual stun film...,positive


In [9]:

nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
#lemmatizing the text data

dp1 = {}

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    #text = [wordnet_lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)]
    lemmatized_text = []
    for word in text.split():
        if word not in dp1:
            dp1[word] = wordnet_lemmatizer.lemmatize(word)
            lemmatized_text.append(dp1[word])
        else:
            lemmatized_text.append(dp1[word])
    return " ".join(lemmatized_text)
df['review'] = df['review'].map(lemmatize)
df.head()

,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic there famili littl boy jake think there ...,negative
4,petter mattei love time money visual stun film...,positive


In [63]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
def sentence_length(text):
    word_tokens = nltk.word_tokenize(text)
    return len(word_tokens)

data = df[['review','sentiment']]

data['review'] = data['review'].map(sentence_length)


def sent(text):
    if text == "positive":
        return 1
    else:
        return 0
    
data['sentiment'] = data['sentiment'].map(sent)
    
data.head()

,review,sentiment
0,168,1
1,84,1
2,86,1
3,67,0
4,125,1


In [76]:
n_pos = np.sum(data['sentiment'])
n_neg = len(data['sentiment']) - n_pos
print("Statistics of cleaned data ------------->")
print()

print(data.describe())

print()
print(f"Ratio of positives to negatives is",(n_pos/n_neg))

Statistics of cleaned data ------------->

             review     sentiment
count  50000.000000  50000.000000
mean     119.932280      0.500000
std       90.157337      0.500005
min        3.000000      0.000000
25%       64.000000      0.000000
50%       89.000000      0.500000
75%      146.000000      1.000000
max     1429.000000      1.000000

Ratio of positives to negatives is 1.0


# Naive Bayes classifier

In [77]:
# get reviews column from df
reviews = df['review']

# get labels column from df
labels = df['sentiment']

In [79]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoded_labels)
# print(enc.classes_)


[1 1 1 ... 0 0 0]


In [80]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

# train_sentences, test_sentences, train_labels, test_labels

train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews,encoded_labels,test_size = 0.20,random_state = 42,shuffle = True,stratify = encoded_labels)

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [84]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words

#max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences.values).toarray()
train_vocab = vec.get_feature_names()

test_sentences = np.array(test_sentences)

#print(X)

In [18]:
# Use laplace smoothing for words in test set not present in vocab of train set

# laplace the smoothing has been done by building the model itself

In [93]:
# Build the model. Don't use the model from sklearn

memory0 = {}
memory1 = {}

X_filtered_1 = X[train_labels == 1]
X_filtered_2 = X[train_labels == 0]

# funtion for conditional probability

def cond_propab(X,y_train, test_word, label,train_vocab):
    
    if label == 1:
        X_filtered = X_filtered_1
    else:
        X_filtered = X_filtered_2
    denom = X_filtered.shape[0]
    
    try:
        ind = train_vocab.index(test_word)
    except ValueError:
        ind = -1
    
    if ind == -1:
        num = 0
        return float((num + 1)/(denom + 3000))     # Laplace smoothing for the words not in the vocab
    else:
        num = np.sum(X_filtered[:, ind])
        return float((num)/(denom))
    
        

def predict(X,y_train,x_test,train_vocab):
    
    likelyhood = 1.0
    word = nltk.word_tokenize(x_test)
    for w in word:
        
        if w not in memory0:
            memory0[w] = cond = cond_propab(X,y_train,w,0,train_vocab)
        likelyhood = likelyhood*memory0[w]
        
    prior = 0.5
    post0 = likelyhood*0.5
    
    likelyhood = 1.0
    for w in word:
        if w not in memory1:
            memory1[w] = cond = cond_propab(X,y_train,w,1,train_vocab)
        likelyhood = likelyhood*memory1[w]
        
    prior = 0.5
    post1 = likelyhood*0.5
    
    if(post0 > post1):
        return 0
    else:
        return 1
 

In [94]:
# Test the model on test set and report Accuracy

def accuracy(X, y_train, X_test, y_test,train_vocab):
    pred = []
    
    for i in range(X_test.shape[0]):
        p = predict(X,y_train,X_test[i],train_vocab)
        pred.append(p)
    y_pred = np.array(pred)
    acc = np.sum(y_pred == y_test)/y_pred.shape[0]
    
    return acc

In [95]:
acc = accuracy(X,train_labels,test_sentences,test_labels,train_vocab)

In [104]:
print(f"The accuracy of the Naive Bayes model is", acc*100)

The accuracy of the Naive Bayes model is 79.17999999999999


# *LSTM* based Classifier

Use the above train and test splits.

In [123]:
# Hyperparameters of the model
vocab_size = 3000# choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 150# choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [124]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [125]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 100)          300000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_8 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [126]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 117s 103ms/step - loss: 0.3946 - accuracy: 0.8189 - val_loss: 0.3017 - val_accuracy: 0.8735
Epoch 2/5
1125/1125 [==============================] - 120s 106ms/step - loss: 0.2848 - accuracy: 0.8844 - val_loss: 0.2944 - val_accuracy: 0.8750
Epoch 3/5
1125/1125 [==============================] - 120s 107ms/step - loss: 0.2457 - accuracy: 0.9016 - val_loss: 0.3172 - val_accuracy: 0.8545
Epoch 4/5
1125/1125 [==============================] - 121s 107ms/step - loss: 0.2098 - accuracy: 0.9184 - val_loss: 0.3110 - val_accuracy: 0.8708
Epoch 5/5
1125/1125 [==============================] - 121s 108ms/step - loss: 0.1825 - accuracy: 0.9302 - val_loss: 0.3535 - val_accuracy: 0.8625


## Get predictions for random examples

In [130]:
# Calculate accuracy on Test data
'''
prediction = model.predict(test_padded)

'''
# Get probabilities

prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0

labels = (prediction >= 0.5)

# Accuracy : one can use classification_report from sklearn
from sklearn.metrics import classification_report

classification_report(labels, test_labels)

'              precision    recall  f1-score   support\n\n       False       0.83      0.90      0.86      4613\n        True       0.91      0.84      0.87      5387\n\n    accuracy                           0.87     10000\n   macro avg       0.87      0.87      0.87     10000\nweighted avg       0.87      0.87      0.87     10000\n'

In [131]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
print(model.predict(padded))

# Get labels based on probability 1 if p>= 0.5 else 0

prediction = model.predict(padded)
labels = (prediction >= 0.5)

print()

print("Prediction of the sentiment of three reviews : ")

for i in range(3):
    if labels[i] == True:
        x = 'positive'
    else:
        x = 'negative'
    print(sentence[i] + " ------> "  + x)


[[0.58217144]
 [0.47325543]
 [0.53228295]]

Prediction of the sentiment of three reviews : 
The movie was very touching and heart whelming ------> positive
I have never seen a terrible movie like this ------> negative
the movie plot is terrible but it had good acting ------> positive
